In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from dotenv import load_dotenv
import uuid

from prompts import (
    SYSTEM_PROMPT,
    BACKGROUND_ASK_TEMPLATE,
    BACKGROUND_TO_ROLE_TEMPLATE,
    ROLE_DETAIL_TEMPLATE,
    GAP_ANALYSIS_TEMPLATE,
)

load_dotenv()

# 1) LLM 생성
llm = ChatOpenAI(model="gpt-5-mini")

# 2) 하나의 "슈퍼" 시스템 프롬프트 만들기
#    - 여기 안에 1~4단계 행동 규칙을 모두 넣고
#    - 챗봇이 스스로 단계 전환을 판단하도록 지시
FULL_SYSTEM_PROMPT = (
    SYSTEM_PROMPT
    + """

[전체 플로우 운영 규칙]

너는 아래 4단계를 **직접 관리**하면서,
사용자의 말투와 질문 의도를 보고 자연스럽게 다음 단계로 넘어가야 한다.

- "지금은 몇 단계입니다" 같은 말은 절대 하지 말고,
  그냥 자연스러운 상담 대화처럼 이어가라.

1단계) 백그라운드 질문 단계
- 사용자가 자신의 상황을 충분히 말하지 않았으면,
  아래 가이드를 참고하여 1~2개씩 천천히 물어본다.
- 사용자의 대답이 너무 짧으면, 예시를 들면서 조금 더 구체적으로 물어본다.

[백그라운드 질문 가이드]
"""
    + BACKGROUND_ASK_TEMPLATE
    + """

2단계) 직무 추천 단계
- 사용자가 어느 정도 자신의 배경을 이야기했고,
  "저한테 어울리는 직무가 뭐가 있을까요?",
  "무슨 일을 하면 좋을까요?" 같은 질문을 하면
  아래 형식에 맞게 직무들을 정리해서 추천한다.

[직무 추천 시 출력 형식/가이드]
"""
    + BACKGROUND_TO_ROLE_TEMPLATE
    + """

3단계) 선택한 직무 상세 설명 단계
- 사용자가 추천받은 직무 중 하나를 골라서
  "데이터 분석가가 궁금해요", "백엔드 개발자에 대해서 자세히 알려주세요"
  처럼 말하면, 그 직무를 {role_name}이라고 생각하고 아래 템플릿에 맞게 설명한다.
- 사용자가 직무 이름을 정확히 말하지 않더라도,
  직전에 언급된 직무/관심을 보고 합리적으로 추론해도 된다.

[선택 직무 상세 설명 가이드]
"""
    + ROLE_DETAIL_TEMPLATE
    + """

4단계) 갭 분석 + 공부 로드맵 단계
- 사용자가 "앞으로 어떻게 공부하면 좋을까요?",
  "제가 부족한 부분이 뭐가 있을까요?"처럼 물어보면
  지금까지의 대화 내용을 기반으로,
  아래 템플릿 형식에 맞게 역량 갭과 공부 로드맵을 제안한다.
- {retrieved_docs} 부분은, 나중에 RAG로 가져온
  '해당 직무 채용공고/기술스택 요약'이 들어갈 자리라고 생각하고,
  지금은 대화 내용만으로 최대한 합리적으로 작성한다.

[갭 분석/로드맵 가이드]
"""
    + GAP_ANALYSIS_TEMPLATE
    + """

[추가 규칙]

- 사용자가 "잘 모르겠다", "그냥 추천해줘"라고 해도
  부담을 줄여주면서 선택지를 제시해라.
- 단계 이름, '템플릿' 같은 단어는 사용자에게 드러내지 않는다.
- 부족한 정보가 있다고 느껴지면, 먼저 1~2개의 구체적인 질문으로
  정보를 채운 뒤에 답을 만들어라.
"""
)

# 3) 프롬프트 템플릿: system + history + human
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", FULL_SYSTEM_PROMPT),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

# 4) 체인 구성
chain = prompt | llm

# 5) 세션별 히스토리 저장소
store = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 6) RunnableWithMessageHistory 설정
runnable = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

# 7) 세션 ID 하나 생성 (콘솔 기준)
session_id = str(uuid.uuid4())

# 8) 콘솔 루프
if __name__ == "__main__":
    print("💬 IT 진로/커리어 상담 챗봇입니다.")
    print("   편하게 지금 상황부터 말해보세요! (종료: '종료')\n")

    while True:
        user_input = input("👤 나: ").strip()

        if user_input in ["종료", "quit", "exit"]:
            print("👋 종료합니다. 진로 고민 생기면 또 와 주세요!")
            break

        response = runnable.invoke(
            {"input": user_input,
            "role_name": "",    
            "retrieved_docs": "",  
            },
            config={"configurable": {"session_id": session_id}},
        )

        print("🤖 챗봇:", response.content)


💬 IT 진로/커리어 상담 챗봇입니다.
   편하게 지금 상황부터 말해보세요! (종료: '종료')

🤖 챗봇: 안녕하세요! 반갑습니다 :) 몇 가지만 물어볼게요 — 편하게 답해주시면 거기 맞춰서 직무 추천 도와드릴게요.

1) 전공과 학년이 어떻게 되세요?  
2) 지금까지 써본 프로그래밍 언어(예: C, Java, Python 등)와 수준(초급/중급/고급) 알려주세요.  
   - 만약 Python을 썼다면 어떤 라이브러리(pandas, numpy, matplotlib 등)까지 써봤는지도 적어주시면 좋아요.

답해주시면 다음으로 해본 프로젝트나 관심 분야도 물어볼게요.
🤖 챗봇: 좋아요, 컴공 2학년이면 시작하기 좋은 시기예요. 몇 가지만 더 물어볼게요 — 편하게 적어주세요.

1) 지금까지 써본 프로그래밍 언어와 수준(초급/중급/고급)을 알려주세요. 예: C(중급), Java(초급), Python(간단한 스크립트 정도).  
   - Python을 썼다면 pandas/numpy/tensorflow 같은 라이브러리 사용 경험도 적어주세요.

2) 해본 프로젝트나 과제(학교 수업 프로젝트, 개인 프로젝트, 팀 프로젝트 등)가 있으면 간단히 설명해주세요. 예: 웹사이트 제작, 데이터 분석 과제, 알고리즘 문제 풀이 등.

답해주시면 그다음에 관심 분야(데이터/백엔드/프론트/앱/임베디드/게임 등)도 물어볼게요.
🤖 챗봇: 좋아요 — 답변 감사합니다. 주신 정보를 바탕으로 몇 가지 잘 맞는 직무를 추천드릴게요.

1. 사용자 백그라운드 요약
   - 전공/학년: 컴공 2학년  
   - 사용 가능한 언어/도구: Python (pandas 사용 경험)  
   - 해본 프로젝트/경험: 데이터 분석 과제 수행  
   - 관심 분야: 데이터 분석/데이터 관련 분야에 흥미가 있는 것으로 보임

2. 적합해 보이는 직무 추천
   - 직무 이름: 데이터 분석가 (Data Analyst)
     - 이 직무가 하는 일: 데이터를 수집·정리하고 통계/시각화를 통해 인사이트를

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from dotenv import load_dotenv
import uuid


from prompts import (
    SYSTEM_PROMPT,
    BACKGROUND_ASK_TEMPLATE,
    BACKGROUND_TO_ROLE_TEMPLATE,
    ROLE_DETAIL_TEMPLATE,
    GAP_ANALYSIS_TEMPLATE,
)

load_dotenv()

# 1) LLM 생성
llm = ChatOpenAI(model="gpt-5-mini")

# 2) "슈퍼" 시스템 프롬프트 (1~4단계 플로우 전부 내장)
FULL_SYSTEM_PROMPT = (
    SYSTEM_PROMPT
    + """

[전체 플로우 운영 규칙]

너는 아래 4단계를 **직접 관리**하면서,
사용자의 말투와 질문 의도를 보고 자연스럽게 다음 단계로 넘어가야 한다.

- "지금은 몇 단계입니다" 같은 말은 절대 하지 말고,
  그냥 자연스러운 상담 대화처럼 이어가라.

1단계) 백그라운드 질문 단계
- 사용자가 자신의 상황을 충분히 말하지 않았으면,
  아래 가이드를 참고하여 1~2개씩 천천히 물어본다.
- 사용자의 대답이 너무 짧으면, 예시를 들면서 조금 더 구체적으로 물어본다.

[백그라운드 질문 가이드]
"""
    + BACKGROUND_ASK_TEMPLATE
    + """

2단계) 직무 추천 단계
- 사용자가 어느 정도 자신의 배경을 이야기했고,
  "저한테 어울리는 직무가 뭐가 있을까요?",
  "무슨 일을 하면 좋을까요?" 같은 질문을 하면
  아래 형식에 맞게 직무들을 정리해서 추천한다.

[직무 추천 시 출력 형식/가이드]
"""
    + BACKGROUND_TO_ROLE_TEMPLATE
    + """

3단계) 선택한 직무 상세 설명 단계
- 사용자가 추천받은 직무 중 하나를 골라서
  "데이터 분석가가 궁금해요", "백엔드 개발자에 대해서 자세히 알려주세요"
  처럼 말하면, 그 직무를 {role_name}이라고 생각하고 아래 템플릿에 맞게 설명한다.
- 사용자가 직무 이름을 정확히 말하지 않더라도,
  직전에 언급된 직무/관심을 보고 합리적으로 추론해도 된다.

[선택 직무 상세 설명 가이드]
"""
    + ROLE_DETAIL_TEMPLATE
    + """

4단계) 갭 분석 + 공부 로드맵 단계
- 사용자가 "앞으로 어떻게 공부하면 좋을까요?",
  "제가 부족한 부분이 뭐가 있을까요?"처럼 물어보면
  지금까지의 대화 내용을 기반으로,
  아래 템플릿 형식에 맞게 역량 갭과 공부 로드맵을 제안한다.
- {retrieved_docs} 부분은, 나중에 RAG로 가져온
  '해당 직무 채용공고/기술스택 요약'이 들어갈 자리라고 생각하고,
  지금은 대화 내용만으로 최대한 합리적으로 작성한다.

[갭 분석/로드맵 가이드]
"""
    + GAP_ANALYSIS_TEMPLATE
    + """

[추가 규칙]

- 사용자가 "잘 모르겠다", "그냥 추천해줘"라고 해도
  부담을 줄여주면서 선택지를 제시해라.
- 단계 이름, '템플릿' 같은 단어는 사용자에게 드러내지 않는다.
- 부족한 정보가 있다고 느껴지면, 먼저 1~2개의 구체적인 질문으로
  정보를 채운 뒤에 답을 만들어라.
"""
)

# 3) 프롬프트 템플릿: system + history + human
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", FULL_SYSTEM_PROMPT),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

# 4) 체인 구성
chain = prompt | llm

# 5) 세션별 히스토리 저장소
_store: dict[str, ChatMessageHistory] = {}


def get_session_history(session_id: str) -> ChatMessageHistory:
    """세션 ID별로 LangChain 대화 히스토리 관리"""
    if session_id not in _store:
        _store[session_id] = ChatMessageHistory()
    return _store[session_id]


# 6) RunnableWithMessageHistory 설정
runnable = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)


# 7) 외부에서 사용할 함수: session_id + user_input → 모델 응답 문자열
def get_chat_response(
    session_id: str,
    user_input: str,
    role_name: str = "",
    retrieved_docs: str = "",
) -> str:
    """
    Streamlit 등 외부에서 호출할 진입점 함수.

    - session_id: 유저/브라우저마다 고유하게 관리
    - user_input: 사용자가 보낸 채팅 문자열
    - role_name, retrieved_docs: 나중에 RAG/직무선택 연동 시 확장용 (지금은 빈값 가능)
    """
    response = runnable.invoke(
        {
            "input": user_input,
            "role_name": role_name,
            "retrieved_docs": retrieved_docs,
        },
        config={"configurable": {"session_id": session_id}},
    )
    return response.content



def create_session_id() -> str:
    return str(uuid.uuid4())
